In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
fig_DIR = "../figs/player_analysis/"
aio_DIR = "../data/all_in_one_metrics/"
misc_DIR = "../data/misc/"

In [ ]:
from nba_api.stats.endpoints import playerindex, leaguedashplayerstats, leaguedashplayerbiostats

In [ ]:
dfa = []
for year in tqdm(range(1996,2024)):
    season_str = str(year) + "-" + str(year+1)[-2:]
    time.sleep(0.6)
    stats = leaguedashplayerbiostats.LeagueDashPlayerBioStats(
        league_id="00", 
        per_mode_simple="Totals", 
        season_type_all_star="Regular Season", 
        season=season_str
    )
    df1 = stats.get_data_frames()[0]
    df1.columns = map(str.lower, df1.columns)
    time.sleep(0.6)
    stats = leaguedashplayerstats.LeagueDashPlayerStats(
        measure_type_detailed_defense="Base",
        per_mode_detailed="Totals",
        season_type_all_star="Regular Season",
        season=season_str,
        player_position_abbreviation_nullable="C"
    )
    df2 = stats.get_data_frames()[0]
    df2.columns = map(str.lower, df2.columns)
    df1 = df1.rename(columns={"player_id":"pID"})
    df2 = df2.rename(columns={"player_id":"pID"})
    df1 = df1[["pID","player_height_inches","player_weight"]]
    df2 = df2[["pID","player_name","min","pts","blk"]]
    df3 = pd.merge(df2,df1,on="pID")
    df3["season"] = year + 1
    dfa.append(df3)


In [ ]:
df = pd.concat(dfa)
idx = df[df['player_weight'] == ''].index
df.loc[idx,'player_weight'] = 0
df["player_weight"] = df["player_weight"].astype("int")
df["height_min"] = df["player_height_inches"]*df["min"]
df["weight_min"] = df["player_weight"]*df["min"]
df = df.drop(idx)

In [ ]:
dfs = df.groupby("season")[["min","height_min","weight_min"]].sum().reset_index()
dfs["height"] = round(dfs["height_min"]/dfs["min"],1)
dfs["weight"] = round(dfs["weight_min"]/dfs["min"],1)

In [ ]:
p = (
    ggplot(dfs)
    + aes(x="season", y = "height")
    + geom_point()
    + geom_line(size=1)
    + theme_idv
    + theme(
        plot_margin=0.025,
    )
    + labs(
        x="Season",
        y="Height (inches)",
        title=f"Average Height of Centers From 1996-97 NBA Season",
        subtitle=f"Average shown here is Weighted Average by minutes played",
        caption="@SravanNBA | source:nba.com/stats",
    )
)
p.save(fig_DIR + f"center_avg_height.png", dpi=300, verbose=False)
p

In [ ]:
p = (
    ggplot(dfs)
    + aes(x="season", y = "weight")
    + geom_line(size=1)
    + theme_idv
    + theme(
        plot_margin=0.025,
    )
    + labs(
        x="Season",
        y="Weight (lb)",
        title=f"Average Weight of Centers From 1996-97 NBA Season",
        subtitle=f"Average shown here is Weighted Average by minutes played",
        caption="@SravanNBA | source:nba.com/stats",
    )
)
p.save(fig_DIR + f"center_avg_weight.png", dpi=300, verbose=False)
p

In [ ]:
dfd = pd.read_csv(aio_DIR + "NBA_DARKO_History.csv")
dfd = dfd.rename(columns={"idPlayerNBA":"pID"})
dfd = dfd[["pID","season","dpm","o_dpm","d_dpm"]]

In [ ]:
dfm = pd.merge(df,dfd,on=["pID","season"])

In [ ]:
dfm.to_csv(misc_DIR + "To_Matt_Issa.csv")

In [ ]:
dfg = dfm.groupby("season")

In [ ]:
dfb = []
for year in range(1997,2025):
    dfg1 = dfg.get_group(year).nlargest(10,"d_dpm").reset_index(drop=True)
    dfb.append(dfg1)
dfb = pd.concat(dfb)
dfbs = dfb.groupby("season")[["player_height_inches","player_weight"]].mean().reset_index()

In [ ]:
p = (
    ggplot(dfbs)
    + aes(x="season", y = "player_weight")
    + geom_point()
    + geom_line(size=1)
    + scale_x_continuous(breaks=range(1996,2025,4))
    + theme_idv
    + theme(
        plot_margin=0.025,
    )
    + labs(
        x="Season",
        y="Weight (lb)",
        title=f"Average Weight of Top 10 Centers",
        subtitle=f"From 1996-97 NBA Season | Top 10 Centers in DARKO Def-DPM",
        caption="@SravanNBA | source:nba.com/stats,darko.app",
    )
)
p.save(fig_DIR + f"center_ddpm10_weight.png", dpi=300, verbose=False)
p

In [ ]:
p = (
    ggplot(dfbs)
    + aes(x="season", y = "player_height_inches")
    + geom_point()
    + geom_line(size=1)
    + scale_x_continuous(breaks=range(1996,2025,4))
    + theme_idv
    + theme(
        plot_margin=0.025,
    )
    + labs(
        x="Season",
        y="Height (inches)",
        title=f"Average Height of Top 10 Centers",
        subtitle=f"From 1996-97 NBA Season | Top 10 Centers in DARKO Off-DPM",
        caption="@SravanNBA | source:nba.com/stats,darko.app",
    )
)
p.save(fig_DIR + f"center_odpm10_height.png", dpi=300, verbose=False)
p